In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import tensorflow as tf
import os
import numpy as np

2023-09-05 15:33:30.914379: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-05 15:33:31.641649: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [4]:
sequences_data = [
    [1, 2, 3],
    [4, 5],
    [6, 7, 8, 9],
    [1,7],
    [4,5,6,7,8,9,9],
    [9,9,9,9,9,9,9,9,9]
]

labels_data = []
for i in sequences_data:
    labels_data.append(np.random.rand((len(i)-1)*6))

sequences_ragged = tf.ragged.constant(sequences_data, dtype=tf.int32)
labels_ragged = tf.ragged.constant(labels_data, dtype=tf.float64)

dataset = tf.data.Dataset.from_tensor_slices(sequences_ragged)
labels_dataset = tf.data.Dataset.from_tensor_slices(labels_ragged)

2023-09-05 15:33:33.278377: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 42408 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:81:00.0, compute capability: 8.6


In [13]:
def f_n(x,y):
    return x, y

In [17]:
d = tf.data.Dataset.from_tensor_slices((sequences_ragged, labels_ragged))
d = d.map(f_n).padded_batch(batch_size=2)

In [5]:
batched_dataset = dataset.map(lambda x: x).padded_batch(batch_size=2)
batched_dataset_labels = labels_dataset.map(lambda x: x).padded_batch(batch_size=2)

In [6]:
merged_dataset = tf.data.Dataset.zip((batched_dataset, batched_dataset_labels))

In [7]:
from prosit_t.models.test_model import TestModel
from dlomix.losses import masked_spectral_distance

In [8]:
test = TestModel(32)

In [9]:
test.compile(loss=masked_spectral_distance, optimizer='adam')

In [10]:
test.fit(merged_dataset, epochs=10)

Epoch 1/10


TypeError: in user code:

    File "/cmnfs/home/l.mamisashvili/miniconda3/envs/prosit-t/lib/python3.9/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/cmnfs/home/l.mamisashvili/miniconda3/envs/prosit-t/lib/python3.9/site-packages/dlomix/losses/intensity.py", line 23, in masked_spectral_distance  *
        true_norm = K.l2_normalize(pred_masked, axis=-1)
    File "/cmnfs/home/l.mamisashvili/miniconda3/envs/prosit-t/lib/python3.9/site-packages/keras/src/backend.py", line 5979, in l2_normalize
        return tf.linalg.l2_normalize(x, axis=axis)

    TypeError: Failed to convert elements of tf.RaggedTensor(values=Tensor("masked_spectral_distance/control_dependency_6:0", shape=(None, 32), dtype=float32), row_splits=Tensor("masked_spectral_distance/RowPartitionFromUniformRowLength_3/control_dependency:0", shape=(None,), dtype=int64)) to Tensor. Consider casting elements to a supported type. See https://www.te